In [5]:
using Unitful #https://painterqubits.github.io/Unitful.jl/stable/
# quantity * @u_str("unit abbreviation") 
using Symbolics #https://symbolics.juliasymbolics.org/dev/
# cite https://doi.org/10.48550/arXiv.2105.03949
using Latexify
using Test
# 1 * @u_str("mA") is 1 milliamp
function power_efficiency(power_in, resistance, seebeck, delta_t)
        voltage = seebeck * delta_t
        return uconvert(Unitful.NoUnits, (voltage ^ 2) / (resistance * power_in))
end
P, R, S, ΔT = @variables P, R, S, ΔT
output = (S * ΔT)^2 / (R * P)
print(
Test.@test power_efficiency(P * @u_str("W"), R * @u_str("Ω"), 
                            S * @u_str("V") / @u_str("K"), ΔT * @u_str("K")) - output == 0
)

Test Passed

All values use units composed of kilograms, meters, and seconds.

In [59]:
using PhysicalConstants
import PhysicalConstants.CODATA2018:c_0
using UnitfulAstro:Lsun,Rsun,AU,GMsun
L_sun, R_sun = uconvert(@u_str("W"), 1*Lsun), uconvert(@u_str("m"), 1*Rsun)
x_0, GM_sun = uconvert(@u_str("m"), 1*AU), uconvert(@u_str("m")^3 / @u_str("s")^2, 1*GMsun)
[x*1 for x in [L_sun, R_sun, x_0, GM_sun, c_0]]

5-element Vector{Quantity{Float64}}:
            3.828e26 W
             6.957e8 m
      1.495978707e11 m
 1.3271244e20 m^3 s^-2
   2.99792458e8 m s^-1

In [ ]:
grav_acc = GM_sun / x^2
P_sun = L_sun * area / (4*np.pi* x**2)

In [61]:
v, c, m = @variables v, c, m
γ = 1 / sqrt(1 - v^2 / c^2)
KE = (γ - 1) * m * c^2
KE

m*(c^2)*(1 / sqrt(1 + (-(v^2)) / (c^2)) - 1)

In [73]:
gamma(v, c) = 1 / sqrt(1 - v^2 / c^2)
kinetic_energy(v, m, c) = (gamma(v, c) - 1) * m * c^2
print(Test.@test kinetic_energy(v, m, c ) - KE == 0)

Test Passed

In [79]:
kinetic_energy(BigFloat(10)* @u_str("m")/@u_str("s"), 1*@u_str("kg"), c_0)

50.00000000000004172437710201073175070569566477939694833199009616528698564728937 kg m^2 s^-2

In [ ]:
solar_power(x, area) =  L_sun * area / (4*np.pi* x**2)
grav_acc(x) = GM_sun / x**2

\begin{align}
T = \left(\frac{1}{\sqrt{1-(v^2/c^2)}}-1\right)m_0c^2\\
v = c\sqrt{1-\left({\frac{T}{m_0c^2}+1}\right)^2}\\
\Delta v = c\left(\sqrt{1-\left({\frac{T_1}{m_0c^2}+1}\right)^2}-
\sqrt{1-\left({\frac{T_0}{m_0c^2}+1}\right)^2}\right)
\end{align}

In [80]:
v_from_KE(T, m, c) = c*(sqrt(1 - (T / (m * c^2) + 1)))

v_from_KE (generic function with 1 method)

In [82]:
function increment(x_old, v_old, time_step, area, mass)
  x_new = x_old + v_old*time_step
  KE_old = kinetic_energy(v_old, mass, c_0)
  solar_push = solar_power(x_old, area)
  Δv_light = v_from_KE(KE_new, mass, c_0) - v_from_KE(KE_old, mass, c_0)
  Δv_grav = v_old + grav_acc(x_old)*time_step
  v_new = v_old + Δv_light + Δv_grav
  return [x_new, v_new]
end

increment (generic function with 1 method)